In [635]:
import numpy as np

In [636]:
def fit(X_train,Y_train):
    #First i want to make a dictionary
    data_dict={}
    #Then now i want to Find all the unique classes in my target for instance here in my data 0,1,2 are my classes...
    #Now let me find all unique classes this act as my keys for further processing....
    unique_target_names=np.unique(Y_train)
    for i in unique_target_names:
        data_dict[i]={}   #----->This results data_dict[0]--->{}
        #Now i want to find all the feature names in my datasets....
        #print(X_train.columns)------>It gives the Range Index of start and end value which is diificult to iterate..
        filter_X_train=X_train[Y_train==i]
        data_dict[i]['total_class']=len(filter_X_train)
        for feature in X_train.columns:
            unique_column_values=np.unique(X_train[feature])
            #print("Unique_column_values are {}".format(unique_column_values))
            #print("Guys this is for the unique_column {}".format(i))
            data_dict[i][feature]={}
            
            for unique_value in unique_column_values:
                
                count=(filter_X_train[feature]==unique_value)
                count=(filter_X_train[feature][count].count())
                #print(count)
                data_dict[i][feature][unique_value]=count
              #  print("I want to give the count of each different values in the columnn {} and count is {}".format(unique_value,count))
   
    data_dict['total_training']=len(Y_train)
    return data_dict

In [729]:
def find_best_probability_class(data_dict,datapoint):
    #Now i want to find the best class to do that i want to find the probabilities over all classes and find the highest probability class..
    best_class=-100
    best_probab=-1000
    find_all_target_classes=data_dict.keys()
    #print(find_all_target_classes)
    output=1
    #Now i want to apply the bayes theorem i.e p(y/x)=p(x/y)*p(y)/p(x).. here i ignore p(x) because in testing there is highprobabilty that i wont see the point
    for i in find_all_target_classes:
        output=1
        count=0
        if(i is 'total_training'):
            continue
        output=output*data_dict[i]['total_class']/data_dict['total_training']
        
        feature_keys=data_dict[i].keys()
        for feature in feature_keys:
            if(feature is 'total_class'):
                continue
            xd=len(data_dict[i][feature].keys())
            ratio=(data_dict[i][feature][datapoint[feature]]+1)/(data_dict[i]['total_class']+xd)
            output=output*ratio
        #print("The output for class {} is {}".format(i,output))  
        if(output>best_probab):
            best_probab=output
            best_class=i
            count=count+1
            
    #print("Finally The best class is {} and probabilty now is:{} and count is {}".format(best_class,best_probab,count))
    return best_class

In [730]:
def predict(data_dict,X_test):
    predict_class_lst=[]
    for i in range(len(X_test)):
        datapoint=X_test.iloc[i]
        present_class=find_best_probability_class(data_dict,datapoint)
        predict_class_lst.append(present_class)
    return predict_class_lst
        

In [731]:
def mapper(lst,mean_value):
    for i in range(len(lst)):
        if(lst[i]<0.25*mean_value):
            lst[i]=0
        elif(lst[i]>0.25*mean_value and lst[i]<mean_value):
            lst[i]=1
        elif(lst[i]>mean_value and lst[i]<1.5*mean_value):
            lst[i]=2
        elif(lst[i]>=1.5*mean_value):
            lst[i]=3
    lst=np.array(lst,dtype='int')
    return lst

In [732]:
from sklearn.datasets import load_iris
import pandas as pd
data=load_iris()
panda_data=pd.DataFrame(data.data)
print(data.target)
length=len(panda_data.columns)
for i in range(length):
    lst=panda_data[i]
    lst=np.array(lst)
    mean_value=np.mean(lst)
    panda_data[i]=mapper(lst,mean_value)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [733]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(panda_data,data.target,test_size=0.3,random_state=2)
X_train=pd.DataFrame(X_train)
X_train.index=np.arange(len(X_train))
X_test.index=np.arange(len(X_test))
Y_train=pd.Series(Y_train)
Y_train.value_counts()

2    37
1    35
0    33
dtype: int64

In [734]:
X_train.columns=list('abcd')
X_test.columns=list('abcd')

In [735]:
dictionary=fit(X_train,Y_train)
dictionary

{0: {'a': {1: 33, 2: 0},
  'b': {1: 4, 2: 29},
  'c': {1: 33, 2: 0, 3: 0},
  'd': {0: 23, 1: 10, 2: 0, 3: 0},
  'total_class': 33},
 1: {'a': {1: 17, 2: 18},
  'b': {1: 28, 2: 7},
  'c': {1: 5, 2: 30, 3: 0},
  'd': {0: 0, 1: 9, 2: 25, 3: 1},
  'total_class': 35},
 2: {'a': {1: 5, 2: 32},
  'b': {1: 26, 2: 11},
  'c': {1: 0, 2: 24, 3: 13},
  'd': {0: 0, 1: 0, 2: 4, 3: 33},
  'total_class': 37},
 'total_training': 105}

In [741]:
ans=predict(dictionary,X_test)
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(Y_test,ans))
print(classification_report(ans,Y_test))

[[17  0  0]
 [ 1 14  0]
 [ 0  0 13]]
             precision    recall  f1-score   support

          0       1.00      0.94      0.97        18
          1       0.93      1.00      0.97        14
          2       1.00      1.00      1.00        13

avg / total       0.98      0.98      0.98        45

